In [1]:
import murraylab_tools.echo as mt_echo
import os.path

# Echo Setup - Salt Calibration Example

This notebook shows an example of how to programmatically create a picklist for an experiment using the murraylab_tools's Echo subpackage. The experiment in question is a TX-TL buffer calibration experiment. Whenever we make a fresh batch of TX-TL, we make a new buffer solution with custom MgCl and KCl concentrations optimized for maximum production from a GFP plasmid. To find those concentrations, we'll have to run TX-TL reactions with a range of MgCl and KCl concentrations. 

First, we choose where to put our files (in this case, in a "salt calibration" folder).

In [2]:
salt_calibration_input_folder = os.path.join("salt_calibration","inputs")
salt_calibration_output_folder = os.path.join("salt_calibration","outputs")
plate_file = os.path.join(salt_calibration_output_folder, "salt_calibration_plate.dat")
output_name = os.path.join(salt_calibration_output_folder, "salt_calibration_experiment")

Next, we define input materials. Even though this is a TX-TL experiment, we can't use a standard `TXTLMasterMix` class to define a master mix, because there's no buffer solution for this extract yet -- we'll need to add each buffer component to the master mix, leaving out only MgCl and KCl salts. We'll use a `MixtureMaterial` object to represent this master mix.

In [12]:
# DNA concentration calculation
pos_control_gfp_stock_ng_per_uL = 19
pos_control_gfp_length = 3202
pos_control_stock_nm = pos_control_gfp_stock_ng_per_uL*10**6/(660*pos_control_gfp_length)

# Define materials
txtl            = mt_echo.EchoSourceMaterial("TX-TL extract", 1, units = "x") # x
PEG             = mt_echo.EchoSourceMaterial("PEG-8000", 40, units = "x") # x
energy_solution = mt_echo.EchoSourceMaterial("Energy Solution", 14, units = "x") # x
amino_acids     = mt_echo.EchoSourceMaterial("Amino Acids", 6, units = "mM") # mM
water           = mt_echo.EchoSourceMaterial("Water", 1, units = "x") # x
gfp_plasmid     = mt_echo.EchoSourceMaterial("pBEST-GFP", pos_control_stock_nm, units = "nM") # nM

# Define master mix
master_mix = mt_echo.MixtureMaterial(name = "Master Mix", recipe_excess=1.1)
master_mix.add_material(txtl, 0.33)
master_mix.add_material(PEG, 2)
master_mix.add_material(energy_solution, 1)
master_mix.add_material(amino_acids, 1.5)
master_mix.add_material(gfp_plasmid, 1)

print("Master mix contains:")
for material, concentration, dimensionality in master_mix.materials:
    print(f"\t{material} at concentration {concentration}, dimensionality = {dimensionality}")
print("fill material: " + str(master_mix.fill_material))
# print(master_mix.text_recipe())

Master mix contains:
	TX-TL extract (1.000 x) at concentration 0.33, dimensionality = final_concentration
	PEG-8000 (40.000 x) at concentration 2, dimensionality = final_concentration
	Energy Solution (14.000 x) at concentration 1, dimensionality = final_concentration
	Amino Acids (6.000 mM) at concentration 1.5, dimensionality = final_concentration
	pBEST-GFP (8.991 nM) at concentration 1, dimensionality = final_concentration
fill material: None


We'll also need to add salts. Here the potassium glutamate (k_salt) is diluted 3X to make it easier to dispense via Echo. We use 400 mM magnesium glutamate to ensure there is enough room in the reaction to test the desired conditions, but we can also use 200 mM or 100 mM Mg-glutamate if that's all there is. In that case, we might not be able to test high concentrations of K-glutamate.

In [9]:
# Final concentrations
mg_concs = [2, 4, 6, 8, 10, 12]
k_concs = [40, 60, 80, 100, 120, 140]

# Salt materials
mg_salt = mt_echo.EchoSourceMaterial("Mg", 400, units = "mM")
k_salt = mt_echo.EchoSourceMaterial("K", 3000/3, units = "mM")

Finally, we create an `EchoRun` object, and add our materials. Each well should contain:
* Master mix, including buffer components.
* MgCl at some concentration.
* KCl at some concentration.
* Water to fill to 10 uL. 

In [10]:
# Build an EchoRun object
txtl_plate = mt_echo.SourcePlate(filename = plate_file)
txtl_echo_calculator = mt_echo.EchoRun(plate = txtl_plate)

# Add varying concentrations of Mg-glutamate and K-glutamate in grid starting at destination well B2
txtl_echo_calculator.dilution_series(mg_salt, k_salt, mg_concs, k_concs, "B2")

# Add master mix to all wells, fill remaining volume with water
txtl_echo_calculator.add_material_to_all(master_mix) 
txtl_echo_calculator.fill_all_wells_with(water)

# Create picklist (Echo input CSV file)
txtl_echo_calculator.write_picklist(output_name)

This will produce a picklist in "salt_calibration/salt_calibration_experiment_EchoInput" and a human-readable instruction file in "salt_calibration/salt_calibration_experiment_experiment_overview.txt". Here are the human-readable instructions:

In [11]:
salt_filename = os.path.join("salt_calibration","outputs", "salt_calibration_experiment_experiment_overview.txt")
with open(salt_filename,'r') as salt_file:
    for line in salt_file:
        print(line, end = "")

Materials used:

Mg:
	stock concentration: 400.00 mM
	total volume: 27.30 uL
K:
	stock concentration: 1000.00 mM
	total volume: 53.40 uL
Master Mix:
	Stock Concentration: 1 
	Total Volume: 439500.00
		178.47 uL TX-TL extract (1.000 x)
		27.04 uL PEG-8000 (40.000 x)
		38.63 uL Energy Solution (14.000 x)
		135.20 uL Amino Acids (6.000 mM)
		60.15 uL pBEST-GFP (8.991 nM)
Water:
	stock concentration: 1.00 x
	total volume: 49.80 uL

On the source plate:
	Add 27.3 uL of Mg in well: A02
	Add 53.4 uL of K in well: A04
	Add 65.0 uL of Master Mix in wells: A14, A15, A16, A17, A18, A19
	Add 49.5 uL of Master Mix in well: A20
	Add 49.8 uL of Water in well: A22